__PREPARING DATASET__

In [105]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.model_selection import RandomizedSearchCV

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [224]:
FEATURES_DIR = 'F:/src/features'
# coocc_features_fp = os.path.join(FEATURES_DIR, 'f_coocc_288.txt')
aec2d_features_fp = os.path.join(FEATURES_DIR, 'f_aec2d_new_half_size_0.2_256.txt')
aec2d_test_features_fp = os.path.join(FEATURES_DIR, 'f_aec2d_new_half_size_0.2_256_test.txt')

In [225]:
#проверяем пересечение множеств id (методы могли упасть на разных данных)
# coocc_features = np.loadtxt(coocc_features_fp,delimiter=',', dtype=str)
# print(coocc_features.shape)
# coocc_ids = set(coocc_features[:,2])

aec2d_features = np.loadtxt(aec2d_features_fp, delimiter=',', dtype=str)
aec2d_test_features = np.loadtxt(aec2d_test_features_fp, delimiter=',', dtype=str)
print(aec2d_features.shape)
print(aec2d_test_features.shape)
aec2d_ids = set(aec2d_features[:,2])
aec2d_test_ids = set(aec2d_test_features[:,2])

#assert coocc_ids == aec_ids, 'patients sets differs!'

(3617, 35)
(915, 35)


In [218]:
gender_age = [gender+age for gender, age in zip(aec2d_features[:,0], aec2d_features[:,1])]
u,c = np.unique(gender_age, return_counts=True)
hist = np.histogram(c, bins=u)
print(u,c)
GROUP_SIZE = min(c)
print('minimal age-sex group size:', GROUP_SIZE)

['F18' 'F19' 'F20' 'F21' 'F22' 'F23' 'F38' 'F39' 'F40' 'F41' 'F42' 'F43'
 'M18' 'M19' 'M20' 'M21' 'M22' 'M23' 'M38' 'M39' 'M40' 'M41' 'M42' 'M43'] [178 158 138 179 226  47 110 141 181 271 179 132 172  98 174 172  93  93
 121 150 174 125 186 119]
minimal age-sex group size: 47


In [13]:
# fem_idx = np.argwhere(aec2d_features[:,0] == 'F')
# m_idx = np.argwhere(aec2d_features[:,0] == 'M')
# females = np.squeeze(aec2d_features[fem_idx,:])
# males = np.squeeze(aec2d_features[m_idx,:])
# print('FEMALE:\n', females.shape)
# print('\nMALE:\n', males.shape)

# N = int(GROUP_SIZE*0.9)
# print('train group size', N)
# n = GROUP_SIZE - N
# train_idx = []
# test_idx = []
# for age in ['18', '19', '20', '21', '22', '23', '38', '39', '40', '41', '42', '43']:
#     for arr in [females, males]:
#         tr_f = np.squeeze(arr[np.argwhere(arr[:,1] == age)])
#         print(age, tr_f.shape)
#         train_idx += list(tr_f[:N, 2])
#         test_idx += list(tr_f[N:N+n, 2])
# print('train idx', len(train_idx))
# print('test idx', len(test_idx))

FEMALE:
 (12133, 131)

MALE:
 (12851, 131)
train group size 622
18 (959, 131)
18 (995, 131)
19 (1122, 131)
19 (1051, 131)
20 (890, 131)
20 (1220, 131)
21 (1243, 131)
21 (1261, 131)
22 (1147, 131)
22 (1012, 131)
23 (692, 131)
23 (881, 131)
38 (990, 131)
38 (1182, 131)
39 (1058, 131)
39 (988, 131)
40 (1124, 131)
40 (1123, 131)
41 (1152, 131)
41 (1262, 131)
42 (991, 131)
42 (1048, 131)
43 (765, 131)
43 (828, 131)
train idx 14928
test idx 1680


In [226]:
ROOT_DIR = 'F:/' #TODO: change
OUT_DIR = os.path.join(ROOT_DIR, 'checkpoints//aec')
data_path = os.path.join(ROOT_DIR, 'kt_dataset_dicom')

train_files_list = ROOT_DIR + 'dicom_train_list.txt'
val_files_list = ROOT_DIR + 'val_list_id.txt'
half_val_files_list = ROOT_DIR + 'half_sized_val_list.txt'

train_file = open(train_files_list, 'r')
val_file = open(half_val_files_list, 'r')
train_idx = []
test_idx = []

for line in train_file.readlines():
    line = line.split('_')[1]
    train_idx.append(line.split('/')[0])
    
for line in val_file.readlines():
    line = line.split('_')[1]
    test_idx.append(line.split('/')[0])

In [227]:
print('train idx', len(train_idx))
print('test idx', len(test_idx))
# print(train_idx[0])


train idx 3617
test idx 915


In [228]:
X_train, y_train, X_test, y_test = get_experiment_data(aec2d_features, aec2d_test_features, train_idx, test_idx)

train shape (3617, 32)
test shape (915, 32)


In [229]:
X_test[0]

array([0.24474743, 5.0515637 , 2.842302  , 0.83553135, 0.        ,
       4.4130406 , 3.3205523 , 5.7624326 , 0.        , 5.327799  ,
       2.4392748 , 0.6409592 , 3.569295  , 3.3345418 , 0.        ,
       5.962365  , 5.6844916 , 1.3398347 , 4.285871  , 4.160901  ,
       5.4202533 , 0.7940262 , 2.0789604 , 3.917888  , 2.1439786 ,
       2.5608718 , 2.5224357 , 4.0880823 , 6.326989  , 6.4265146 ,
       6.442129  , 4.9727955 ])

In [210]:
def get_experiment_data(train_features, test_features, train_idx, test_idx):
    X_train = np.zeros(shape=(len(train_idx), train_features.shape[1] - 3))
    print('train shape', X_train.shape)
    unique_train_idx = np.unique(train_idx)
    y_train = np.zeros(shape=len(train_idx))
    for i, pid in enumerate(unique_train_idx):
        k = np.argwhere(train_features[:, 2] == pid)
        for j in k:
            X_train[j[0], :] = np.squeeze(train_features[j[0], 3:])
            y_train[j[0]] = (0 if int(train_features[j[0], 1]) < 24 else 1)

    X_test = np.zeros(shape=(len(test_idx), test_features.shape[1] - 3))
    unique_test_idx = np.unique(test_idx)
    y_test = np.zeros(shape=len(test_idx))
    for i, pid in enumerate(unique_test_idx):
        k = np.argwhere(test_features[:, 2] == pid)
        for j in k:
            X_test[j[0], :] = np.squeeze(test_features[j[0], 3:])
            y_test[j[0]] = (0 if int(test_features[j[0], 1]) < 24 else 1)

    print('test shape', X_test.shape)

    return X_train, y_train, X_test, y_test

In [212]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
    return results['mean_test_score'][np.flatnonzero(results['rank_test_score'] == 1)[0]]


def optimize(X, y, clf, param_dist, name, n_iter_search = 20):
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist, 
                                       scoring='accuracy', n_iter=n_iter_search, cv=5)
    random_search.fit(X, y)
    best_sol = report(random_search.cv_results_)
    print(name, best_sol)
    

def make_experiment(train_features, test_features, train_idx, test_idx):
    X_train, y_train, X_test, y_test = get_experiment_data(train_features, test_features, train_idx, test_idx)

    #KNN
    clf = KNeighborsClassifier()
    param_dist = {"n_neighbors": np.arange(1, 21, 1)}
    optimize(X_train, y_train, clf, param_dist, 'KNN')
    
    #Random forest
    clf = RandomForestClassifier()
    param_dist = {"n_estimators": np.arange(1, 50, 5),
              "max_depth": np.arange(1, 3, 1)}
    optimize(X_train, y_train, clf, param_dist, 'RF')
    
    #AdaBoost
    clf = AdaBoostClassifier()
    param_dist = {"n_estimators": np.arange(1, 200, 5),
                "learning_rate": np.arange(0.1, 1.1, 0.2)}
    optimize(X_train, y_train, clf, param_dist, 'Ada', 50)
        
    #SVM
    clf = SVC()
    param_dist = {"kernel": ['linear', 'poly', 'rbf'],
              "C": [10**(_) for _ in np.arange(-5, 6, 1, dtype=float)],
              "degree": np.arange(1, 7, 1)}
    optimize(X_train, y_train, clf, param_dist, 'SVM', 30)
    

In [230]:
print("\nWORKING ON AEC FEATURES\n")
make_experiment(aec2d_features, aec2d_test_features, train_idx, test_idx)


WORKING ON AEC FEATURES

train shape (3617, 32)
test shape (915, 32)
Model with rank: 1
Mean validation score: 0.492 (std: 0.032)
Parameters: {'n_neighbors': 20}

Model with rank: 2
Mean validation score: 0.490 (std: 0.026)
Parameters: {'n_neighbors': 19}

Model with rank: 3
Mean validation score: 0.489 (std: 0.030)
Parameters: {'n_neighbors': 17}

KNN 0.49212917325752886
Model with rank: 1
Mean validation score: 0.526 (std: 0.050)
Parameters: {'n_estimators': 31, 'max_depth': 2}

Model with rank: 2
Mean validation score: 0.519 (std: 0.044)
Parameters: {'n_estimators': 16, 'max_depth': 2}

Model with rank: 3
Mean validation score: 0.518 (std: 0.039)
Parameters: {'n_estimators': 6, 'max_depth': 1}

RF 0.5255985267034992
Model with rank: 1
Mean validation score: 0.530 (std: 0.051)
Parameters: {'n_estimators': 11, 'learning_rate': 0.7000000000000001}

Model with rank: 2
Mean validation score: 0.523 (std: 0.056)
Parameters: {'n_estimators': 6, 'learning_rate': 0.5000000000000001}

Model w

In [ ]:
print("\nWORKING ON CO-OCCURENCES FEATURES\n")
make_experiment(coocc_features, train_idx, test_idx)